In [1]:
import torch
import torch.nn as nn

import pandas as pd
import json

from torchvision import models, transforms

from magic_dataset import MagicDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Specify paths

In [3]:
part=1

In [4]:
path_to_json_test = r'data\images_part1_test_public.json'
path_to_bbox_dir_test = r'data\images_part1_test_bboxes\images'
path_to_bbox_csv_test = r'data\images_part1_test_bboxes\bboxes_data.csv'

In [5]:
path_to_json_train = r'data\reference_images_part1.json'
path_to_bbox_dir_train = r'data\reference_images_part1_bboxes\images'
path_to_bbox_csv_train = r'data\reference_images_part1_bboxes\bboxes_data.csv'

In [6]:
with open(path_to_json_test, 'r') as f:
    json_data_test = json.load(f)

with open(path_to_json_train, 'r') as f:
    json_data_train = json.load(f)

bbox_df_test = pd.read_csv(path_to_bbox_csv_test)
bbox_df_train = pd.read_csv(path_to_bbox_csv_train)

In [7]:
annotations_test = pd.DataFrame(json_data_test['annotations'])
annotations_train = pd.DataFrame(json_data_train['annotations'])
annotations_test

,id,image_id,category_id,area,bbox,iscrowd,occluded,attributes,group_id
0,2987,156,None,8859,"[282.21, 253.88, 59.6, 169.87]",0,False,{'occluded': ['Not-Occluded']},0
1,2988,156,None,2175,"[347.91, 229.06, 93.37, 69.38]",0,False,{'occluded': ['Not-Occluded']},0
2,2989,156,None,2109,"[340.34, 265.88, 30.12, 73.62]",0,False,{'occluded': ['Not-Occluded']},0
3,2990,156,None,2135,"[344.22, 203.12, 98.6, 30]",0,False,{'occluded': ['Not-Occluded']},0
4,2991,157,None,8017,"[381.45, 241.5, 106.94, 162.25]",0,True,{'occluded': ['Occluded']},0
...,...,...,...,...,...,...,...,...,...
3328,12444,725,None,1313,"[387.41, 308.9, 62.09, 75.35]",0,True,{'occluded': ['Occluded']},0
3329,12445,725,None,556,"[420.85, 409.12, 37.64, 23.79]",0,True,{'occluded': ['Occluded']},0
3330,12446,725,None,2461,"[245.93, 177.75, 46.24, 105]",0,True,{'occluded': ['Occluded']},0
3331,12447,725,None,2650,"[422.02, 316.5, 79.69, 66.56]",0,True,{'occluded': ['Occluded']},0


### Prepare transformations

In [8]:
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

### Prepare datasets and dataloaders

In [9]:
test_data = MagicDataset(img_dir=path_to_bbox_dir_test, labels_file=path_to_bbox_csv_test, transform=transformations)
train_data = MagicDataset(img_dir=path_to_bbox_dir_train, labels_file=path_to_bbox_csv_train, transform=transformations)
BATCH_SIZE = 256
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)

### Prepare model and load weigths

In [10]:
EMBEDDING_SIZE = 256

model = models.resnet50(pretrained=True)

num_ftrs = model.fc.in_features
model.avgpool = nn.AdaptiveAvgPool2d(output_size=1)

model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.Linear(512, EMBEDDING_SIZE),
)

model = model.to(device)

In [11]:
model.load_state_dict(torch.load(r'model.pth', map_location=torch.device(device)))

<All keys matched successfully>

### Calculate embeddings of images from train set

In [13]:
model.eval()

embeddings = torch.zeros((0, EMBEDDING_SIZE))
labels = torch.zeros((0))

with torch.no_grad():
    for ins, lbs in train_loader:
        ins = ins.to(device)
        lbs = lbs.to(device)
        
        labels = torch.cat([labels.cpu(), lbs.cpu()])
        embeddings = torch.cat([embeddings.cpu(), model(ins).cpu()], dim=0)

### Train KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1, metric='cosine')
knn.fit(embeddings.numpy(), labels.numpy())

KNeighborsClassifier(metric='cosine', n_neighbors=1)

### Calculate embeddings of new images

In [ ]:
embeddings_test = torch.zeros((0, EMBEDDING_SIZE))

with torch.no_grad():
    for ins, _ in test_loader:
        ins = ins.to(device)
        
        embeddings_test = torch.cat([embeddings_test.cpu(), model(ins).cpu()], dim=0)

### Make predictions

In [ ]:
preds = knn.predict(embeddings_test.numpy())

### Save predictions to json

In [ ]:
annotations_test['category_id'] = preds.astype(int)
annotations_test

,id,image_id,category_id,area,bbox,iscrowd,occluded,attributes,group_id
0,2987,156,2,8859,"[282.21, 253.88, 59.6, 169.87]",0,False,{'occluded': ['Not-Occluded']},0
1,2988,156,11,2175,"[347.91, 229.06, 93.37, 69.38]",0,False,{'occluded': ['Not-Occluded']},0
2,2989,156,10,2109,"[340.34, 265.88, 30.12, 73.62]",0,False,{'occluded': ['Not-Occluded']},0
3,2990,156,11,2135,"[344.22, 203.12, 98.6, 30]",0,False,{'occluded': ['Not-Occluded']},0
4,2991,157,2,8017,"[381.45, 241.5, 106.94, 162.25]",0,True,{'occluded': ['Occluded']},0
...,...,...,...,...,...,...,...,...,...
3328,12444,725,3,1313,"[387.41, 308.9, 62.09, 75.35]",0,True,{'occluded': ['Occluded']},0
3329,12445,725,11,556,"[420.85, 409.12, 37.64, 23.79]",0,True,{'occluded': ['Occluded']},0
3330,12446,725,11,2461,"[245.93, 177.75, 46.24, 105]",0,True,{'occluded': ['Occluded']},0
3331,12447,725,11,2650,"[422.02, 316.5, 79.69, 66.56]",0,True,{'occluded': ['Occluded']},0


In [ ]:
json_data_test['annotations'] = annotations_test.to_dict(orient='records')

json_object = json.dumps(json_data_test, indent=4)

with open(f"images_part{part}_test_public_Czarny_Lotos.json", "w") as outfile:
    outfile.write(json_object)